In [252]:
# Import dependencies
import pandas as pd

In [270]:
# Create reousrce file location
purchase_data = ('Resources/purchase_data.csv')

In [271]:
# Create df of the purchase data
pd_df = pd.read_csv(purchase_data, index_col='Purchase ID')
pd_df.head()
# pd_df.dtypes

,SN,Age,Gender,Item ID,Item Name,Price
Purchase ID,,,,,,
0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,Ithergue48,24,Male,92,Final Critic,4.88
3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,Iskosia90,23,Male,131,Fury,1.44


In [272]:
# Displays all unique players to make purchases
player_count = pd_df['SN'].nunique()
player_count
players = pd.DataFrame([player_count], columns=['Total Players'])
players

,Total Players
0,576


In [276]:
# Get number of unique items purchased
num_item = pd_df['Item ID'].nunique()
num_item

# Get the average price per item
avg_price = '${0:.2f}'.format(pd_df['Price'].mean())
avg_price

#Get the total number of purchases
tot_purchases = pd_df['SN'].count()
tot_purchases

# Get the total revenue for the purchases
tot_rev = '${0:.2f}'.format(pd_df['Price'].sum())
tot_rev

# Create a summary table for the Purchasing Analysis
pa_data = [num_item, avg_price, tot_purchases, tot_rev]
pa_columns = ['Number of Unique items', 'Average Price', 'Number of Purchases', 'Total Revenue']
purchase_df = pd.DataFrame([pa_data], columns=pa_columns)
purchase_df

,Number of Unique items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [314]:
# Create new df to group numbers by gender 
gender_df = pd_df.drop_duplicates('SN')
gender_df

new_gender_df = gender_df.groupby('Gender')
new_gender_df.count()

gender_count_uni = new_gender_df['SN'].count()
gender_count_uni

gender_count = pd.DataFrame({'Total Count':gender_count_uni})
gender_count

total_count = gender_count['Total Count'].sum()
total_count

gender_count['Percentage of Players'] = gender_count['Total Count']/total_count
gender_count['Percentage of Players'] = gender_count['Percentage of Players'].map('{0:.2%}'.format)
gender_count

gender_counts = gender_count.sort_values('Total Count', ascending=False)
gender_counts


    



,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [332]:
# Calculating purchases by gender
purch_df = pd_df.groupby('Gender')
purch_df.count()

purch_count = purch_df['SN'].count()
purch_count

avg_purch_price = purch_df['Price'].mean()
avg_purch_price

tot_purch_val = purch_df['Price'].sum()
tot_purch_val

avg_purch_tot_gen = tot_purch_val/gender_count_uni
avg_purch_tot_gen

gender_purchase = pd.DataFrame({'Purchase Count':purch_count, 'Average Purchase Price':avg_purch_price, 'Total Purchase Value':tot_purch_val, 'Average Total Purchase per Person':avg_purch_tot_gen})
gender_purchase = gender_purchase.style.format({'Average Purchase Price':'${:.02f}', 'Total Purchase Price':'${:.02f}', 'Average Total Purchase per Person':'${:.02f}'})
gender_purchase


,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Gender,,,,
Female,113,$3.20,361.940000,$4.47
Male,652,$3.02,1967.640000,$4.07
Other / Non-Disclosed,15,$3.35,50.190000,$4.56


In [457]:
# Creating new df to break down age groups

bins = [0,9,14,19,24,29,34,39,45]
labels = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']
pd_df['Age Summary'] = pd.cut(pd_df['Age'], bins, labels=labels, include_lowest=True)
pd_df

age_df = pd_df.drop_duplicates('SN')
age_df_uni = age_df.groupby('Age Summary')
age_df_uni.count()

age_count_uni = age_df_uni['SN'].count()
age_count = pd.DataFrame({'Total Count':age_count_uni})
age_count

total_age_count = age_count['Total Count'].sum()

age_count['Percentage of Players'] = age_count['Total Count']/total_age_count
age_count['Percentage of Players'] = age_count['Percentage of Players'].map('{0:.2%}'.format)
age_count


,Total Count,Percentage of Players
Age Summary,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [476]:
# Creating Top Spender by total purchase value
top_spend_df = pd_df.groupby('SN')
top_spend_df.count()

top_spend_count = top_spend_df['SN'].count()
top_spend_mean = top_spend_df['Price'].mean()
top_spend_mean
top_spend_total = top_spend_df['Price'].sum()
top_spend_total

top_spend = pd.DataFrame({'Purchase Count':top_spend_count, 'Average Purchase Price':top_spend_mean, 'Total Purchase Value':top_spend_total})
top_spend

top_spenders = top_spend.sort_values('Total Purchase Value', ascending=False)
top_spenders['Average Purchase Price'] = top_spenders['Average Purchase Price'].map('${:.2f}'.format)
top_spenders['Total Purchase Value'] = top_spenders['Total Purchase Value'].map('${:.2f}'.format)
top_spenders.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10
